# Objective

Download and plot 1m DEM data from the USGS 3DEP program's 1m DEM database.

In [1]:
import requests
from zipfile import ZipFile
import io
from io import BytesIO
from osgeo import gdal
urlbase = 'https://prd-tnm.s3.amazonaws.com/StagedProducts/Elevation/1m/Projects/MS_Coastal_Z15_2015/IMG/'
zipfilename = 'USGS_NED_one_meter_x75y350_MS_Coastal_Z15_2015_IMG_2019.zip'
download = requests.get(url=urlbase+zipfilename)

In [2]:
zipfile = ZipFile(BytesIO(download._content))
img_filename = None
xml_filename = None
for f in zipfile.filelist:
    if '.img' in f.filename:
        img_filename = f.filename
    if 'meta.xml' in f.filename:
        xml_filename = f.filename
print("\nGetting DEM from:\n {0}".format(img_filename))
print("\nGetting DEM metadata from:\n {0}".format(xml_filename))


Getting DEM from:
 USGS_NED_one_meter_x75y350_MS_Coastal_Z15_2015_IMG_2019.img

Getting DEM metadata from:
 USGS_NED_one_meter_x75y350_MS_Coastal_Z15_2015_IMG_2019_meta.xml


In [3]:
import xml.etree.ElementTree as ET
f_xml = zipfile.open(xml_filename).read()
root = ET.fromstring(f_xml.decode())
for element in root.iter('*'):
    if element.tag == 'rowcount':
        nrows=int(element.text)
    if element.tag == 'colcount':
        ncols=int(element.text)
#print(nrows,ncols)

In [4]:
# GDAL does not use python exceptions by default
gdal.UseExceptions()
img_file_path = zipfile.extract(img_filename)

In [5]:
# GDAL does not use python exceptions by default
gdal.UseExceptions()
geo = gdal.Open(img_file_path)
drv = geo.GetDriver()
topo = geo.ReadAsArray()

In [6]:
%matplotlib ipympl
from matplotlib import pyplot as plt
fig = plt.figure(figsize=(7,7))
from matplotlib import cm
cutout = topo[(topo.shape[0]*3)//10:((topo.shape[0]*3)//10+topo.shape[0]//3),
              topo.shape[1]//2:(topo.shape[1]//2 + topo.shape[1]//3)]
plt.imshow(cutout, 
           cmap=cm.BrBG_r)
cbar = plt.colorbar(shrink=0.75)
cbar.set_label('meters')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [7]:
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
fig = plt.figure(figsize=(8,8))
ax = fig.gca(projection='3d')
X = np.arange(0.0, float(cutout.shape[0]), 1.0)
Y = np.arange(0.0, float(cutout.shape[1]), 1.0)[::-1]
XX, YY = np.meshgrid(X, Y)
# Plot the surface.
surf = ax.plot_surface(XX, YY, cutout, cmap=cm.BrBG_r,
                       linewidth=0, antialiased=False,
                      rcount=200,ccount=200)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …